In [77]:
import pandas as pd
import math
location = "A" #"A","B","C" or "filename"
gk = 1000
qk = 600
size = 0.9
mf = 1.35
depth = 20
slsverified = False #False or True
pileshape = "square" #"square" or "circular"
piletype = "driven" #"driven" or "bored"
unit_weight = {"FILL":19,"E":15, "S":21, "PILE":24}
nospile = 4 #4, 6, 8, 10


if pileshape == "circular":
    area = math.pi * size ** 2 / 4
    perimeter = math.pi * size

elif pileshape == "square":
    area = size ** 2
    perimeter = 4*size

In [78]:
if location == "A":
    DataFrame = pd.read_csv("AData.csv")
elif location == "B":
    DataFrame = pd.read_csv("BData.csv")
elif location == "C":
    DataFrame = pd.read_csv("CData.csv")
else:
    DataFrame = pd.read_csv(location)
    
DataFrame = DataFrame.fillna(0)
print(DataFrame)

    top  bottom geology  spt  cu
0   0.0     1.0    FILL    0   0
1   1.0     3.0    FILL    0   0
2   3.0    15.0       E    0  23
3  15.0    17.8       E    0  27
4  17.8    20.0       S   66   0
5  20.0    23.0       S  100   0
6  20.0    33.0       S  100   0
7  33.0    36.0       S  100   0
8  36.0    40.0       S  100   0


In [79]:
DataFrame["unit_weight"] = [unit_weight[x] for x in DataFrame["geology"]]
print(DataFrame)

    top  bottom geology  spt  cu  unit_weight
0   0.0     1.0    FILL    0   0           19
1   1.0     3.0    FILL    0   0           19
2   3.0    15.0       E    0  23           15
3  15.0    17.8       E    0  27           15
4  17.8    20.0       S   66   0           21
5  20.0    23.0       S  100   0           21
6  20.0    33.0       S  100   0           21
7  33.0    36.0       S  100   0           21
8  36.0    40.0       S  100   0           21


In [80]:
def thickness(depth,top,bottom):
    if depth >= bottom:
        return(bottom-top)
    elif depth > top:
        return(depth-top)
    else:
        return(0)
DataFrame["thickness"] = [thickness(depth,x,y) for (x,y) in zip(DataFrame["top"],DataFrame["bottom"])]
print(DataFrame)
#print(list(map(lambda x,y: thickness(depth,x,y), DataFrame["top"],DataFrame["bottom"])))

    top  bottom geology  spt  cu  unit_weight  thickness
0   0.0     1.0    FILL    0   0           19        1.0
1   1.0     3.0    FILL    0   0           19        2.0
2   3.0    15.0       E    0  23           15       12.0
3  15.0    17.8       E    0  27           15        2.8
4  17.8    20.0       S   66   0           21        2.2
5  20.0    23.0       S  100   0           21        0.0
6  20.0    33.0       S  100   0           21        0.0
7  33.0    36.0       S  100   0           21        0.0
8  36.0    40.0       S  100   0           21        0.0


In [81]:
q_increment = DataFrame["unit_weight"]*(DataFrame["bottom"]-DataFrame["top"])
DataFrame["q"] = q_increment.cumsum() - q_increment/2
print(DataFrame)

    top  bottom geology  spt  cu  unit_weight  thickness      q
0   0.0     1.0    FILL    0   0           19        1.0    9.5
1   1.0     3.0    FILL    0   0           19        2.0   38.0
2   3.0    15.0       E    0  23           15       12.0  147.0
3  15.0    17.8       E    0  27           15        2.8  258.0
4  17.8    20.0       S   66   0           21        2.2  302.1
5  20.0    23.0       S  100   0           21        0.0  356.7
6  20.0    33.0       S  100   0           21        0.0  524.7
7  33.0    36.0       S  100   0           21        0.0  692.7
8  36.0    40.0       S  100   0           21        0.0  766.2


In [82]:
def self_weight(unit_weight_soil, thickness):
    self_weight = (unit_weight["PILE"] - unit_weight_soil) * area * thickness
    if self_weight > 0 :
        return(self_weight)
    else :
        return(0)

DataFrame["self_weight"] = [self_weight(x,y) for (x,y) in zip(DataFrame["unit_weight"],DataFrame["thickness"])]
print(DataFrame)

    top  bottom geology  spt  cu  unit_weight  thickness      q  self_weight
0   0.0     1.0    FILL    0   0           19        1.0    9.5        4.050
1   1.0     3.0    FILL    0   0           19        2.0   38.0        8.100
2   3.0    15.0       E    0  23           15       12.0  147.0       87.480
3  15.0    17.8       E    0  27           15        2.8  258.0       20.412
4  17.8    20.0       S   66   0           21        2.2  302.1        5.346
5  20.0    23.0       S  100   0           21        0.0  356.7        0.000
6  20.0    33.0       S  100   0           21        0.0  524.7        0.000
7  33.0    36.0       S  100   0           21        0.0  692.7        0.000
8  36.0    40.0       S  100   0           21        0.0  766.2        0.000


In [83]:
def alpha(geology,cu):
    if geology != "E":
        return(0)
    if piletype == "bored":
        alpha = 0.27 + 0.22/(cu/101.325)
        if alpha <= 1:
            return(alpha)
        else:
            return(1)
    elif piletype == "driven": 
        if cu <= 25:
            return(1)
        elif cu <= 75:
            return(1 - 0.5*(cu-25)/50)
        else:
            return (0.5)
        
DataFrame["alpha"] = [alpha(x,y) for (x,y) in zip(DataFrame["geology"],DataFrame["cu"])]
print(DataFrame)

    top  bottom geology  spt  cu  unit_weight  thickness      q  self_weight  \
0   0.0     1.0    FILL    0   0           19        1.0    9.5        4.050   
1   1.0     3.0    FILL    0   0           19        2.0   38.0        8.100   
2   3.0    15.0       E    0  23           15       12.0  147.0       87.480   
3  15.0    17.8       E    0  27           15        2.8  258.0       20.412   
4  17.8    20.0       S   66   0           21        2.2  302.1        5.346   
5  20.0    23.0       S  100   0           21        0.0  356.7        0.000   
6  20.0    33.0       S  100   0           21        0.0  524.7        0.000   
7  33.0    36.0       S  100   0           21        0.0  692.7        0.000   
8  36.0    40.0       S  100   0           21        0.0  766.2        0.000   

   alpha  
0   0.00  
1   0.00  
2   1.00  
3   0.98  
4   0.00  
5   0.00  
6   0.00  
7   0.00  
8   0.00  


In [84]:
def qs_cal(geology,spt,cu,alpha):
    if geology == "E":
        return(alpha*cu)
    elif geology == "S" or "FILL":
        if piletype == "bored":
            return(spt)
        elif piletype == "driven":
            return(2*spt)
        
DataFrame["qs_cal"] = [qs_cal(geology,spt,cu,alpha) for (geology,spt,cu,alpha) 
                       in zip(DataFrame["geology"],DataFrame["spt"],DataFrame["cu"],DataFrame["alpha"])]        
        
print(DataFrame)
        

    top  bottom geology  spt  cu  unit_weight  thickness      q  self_weight  \
0   0.0     1.0    FILL    0   0           19        1.0    9.5        4.050   
1   1.0     3.0    FILL    0   0           19        2.0   38.0        8.100   
2   3.0    15.0       E    0  23           15       12.0  147.0       87.480   
3  15.0    17.8       E    0  27           15        2.8  258.0       20.412   
4  17.8    20.0       S   66   0           21        2.2  302.1        5.346   
5  20.0    23.0       S  100   0           21        0.0  356.7        0.000   
6  20.0    33.0       S  100   0           21        0.0  524.7        0.000   
7  33.0    36.0       S  100   0           21        0.0  692.7        0.000   
8  36.0    40.0       S  100   0           21        0.0  766.2        0.000   

   alpha  qs_cal  
0   0.00    0.00  
1   0.00    0.00  
2   1.00   23.00  
3   0.98   26.46  
4   0.00  132.00  
5   0.00  200.00  
6   0.00  200.00  
7   0.00  200.00  
8   0.00  200.00  


In [85]:
def rs_k(thickness,qs_cal):
    return(qs_cal*perimeter*thickness/mf)
DataFrame["rs_k"] = [rs_k(thickness,qs_cal) for (thickness,qs_cal) in zip(DataFrame["thickness"],DataFrame["qs_cal"])]
print(DataFrame)

    top  bottom geology  spt  cu  unit_weight  thickness      q  self_weight  \
0   0.0     1.0    FILL    0   0           19        1.0    9.5        4.050   
1   1.0     3.0    FILL    0   0           19        2.0   38.0        8.100   
2   3.0    15.0       E    0  23           15       12.0  147.0       87.480   
3  15.0    17.8       E    0  27           15        2.8  258.0       20.412   
4  17.8    20.0       S   66   0           21        2.2  302.1        5.346   
5  20.0    23.0       S  100   0           21        0.0  356.7        0.000   
6  20.0    33.0       S  100   0           21        0.0  524.7        0.000   
7  33.0    36.0       S  100   0           21        0.0  692.7        0.000   
8  36.0    40.0       S  100   0           21        0.0  766.2        0.000   

   alpha  qs_cal     rs_k  
0   0.00    0.00    0.000  
1   0.00    0.00    0.000  
2   1.00   23.00  736.000  
3   0.98   26.46  197.568  
4   0.00  132.00  774.400  
5   0.00  200.00    0.000  
6  

In [86]:
def qb_cal(geology,spt,cu,q):
    if geology == "E":
        return(9*cu+q)
    elif geology == "S" or "FILL":
        if piletype == "driven":
            return(40*spt)
        elif piletype == "bored":
            return(40/3*spt)
DataFrame["qb_cal"] = [qb_cal(geology,spt,cu,q) for (geology,spt,cu,q) 
                     in zip(DataFrame["geology"],DataFrame["spt"],DataFrame["cu"],DataFrame["q"])]
print(DataFrame)

    top  bottom geology  spt  cu  unit_weight  thickness      q  self_weight  \
0   0.0     1.0    FILL    0   0           19        1.0    9.5        4.050   
1   1.0     3.0    FILL    0   0           19        2.0   38.0        8.100   
2   3.0    15.0       E    0  23           15       12.0  147.0       87.480   
3  15.0    17.8       E    0  27           15        2.8  258.0       20.412   
4  17.8    20.0       S   66   0           21        2.2  302.1        5.346   
5  20.0    23.0       S  100   0           21        0.0  356.7        0.000   
6  20.0    33.0       S  100   0           21        0.0  524.7        0.000   
7  33.0    36.0       S  100   0           21        0.0  692.7        0.000   
8  36.0    40.0       S  100   0           21        0.0  766.2        0.000   

   alpha  qs_cal     rs_k  qb_cal  
0   0.00    0.00    0.000     0.0  
1   0.00    0.00    0.000     0.0  
2   1.00   23.00  736.000   354.0  
3   0.98   26.46  197.568   501.0  
4   0.00  132.00  7

In [87]:
#rb_k = rb_cal / mf = qb_cal * base area /mf
def rb_k(depth,top,bottom,qb_cal):
    if top < depth <= bottom:
        return(qb_cal*area/mf)
    else:
        return(0)

DataFrame["rb_k"] = [rb_k(depth,top,bottom,qb_cal) for (top,bottom,qb_cal) 
                     in zip(DataFrame["top"],DataFrame["bottom"],DataFrame["qb_cal"])]
print(DataFrame)

    top  bottom geology  spt  cu  unit_weight  thickness      q  self_weight  \
0   0.0     1.0    FILL    0   0           19        1.0    9.5        4.050   
1   1.0     3.0    FILL    0   0           19        2.0   38.0        8.100   
2   3.0    15.0       E    0  23           15       12.0  147.0       87.480   
3  15.0    17.8       E    0  27           15        2.8  258.0       20.412   
4  17.8    20.0       S   66   0           21        2.2  302.1        5.346   
5  20.0    23.0       S  100   0           21        0.0  356.7        0.000   
6  20.0    33.0       S  100   0           21        0.0  524.7        0.000   
7  33.0    36.0       S  100   0           21        0.0  692.7        0.000   
8  36.0    40.0       S  100   0           21        0.0  766.2        0.000   

   alpha  qs_cal     rs_k  qb_cal    rb_k  
0   0.00    0.00    0.000     0.0     0.0  
1   0.00    0.00    0.000     0.0     0.0  
2   1.00   23.00  736.000   354.0     0.0  
3   0.98   26.46  197.5

In [88]:
def da1c1(rs_k,rb_k,self_weight):
    if 1.35*(gk+self_weight) + 1.5*qk <= 1.0*rs_k + 1.0*rb_k:
        return True
    else:
        return False

def da1c2(rs_k,rb_k,self_weight):
    if piletype == "driven":
        if not slsverified:
            if 1.0*(gk+self_weight) + 1.0*qk <= rb_k/1.7 + rs_k/1.5:
                return True
            else:
                return False
        elif slsverified:
            if 1.0*(gk+self_weight) + 1.0*rk <= rb_k/1.5 + rs_k/1.3:
                return True
            else:
                return False
        
    if piletype == "bored":
        if not slsverified:
            if 1.0*(gk+self_weight) + 1.0*qk <= rb_k/2.0 + rs_k/1.6:
                return True
            else:
                return False
        elif slsverified:
            if 1.0*(gk+self_weight) + 1.0*rk <= rb_k/1.7 + rs_k/1.4 :
                return True
            else:
                return False
        

self_weight_t = DataFrame["self_weight"].sum()        
rs_kt = DataFrame["rs_k"].sum()
rb_kt = DataFrame["rb_k"].sum()

print(da1c1(rs_kt,rb_kt,self_weight_t))
print(da1c2(rs_kt,rb_kt,self_weight_t))

True
True


In [89]:
def runtable():    
    for i in range(1,int(float(DataFrame["bottom"].tail(1))//0.1)):
        depth = i/10
    
        DataFrame["unit_weight"] = [unit_weight[x] for x in DataFrame["geology"]]
        DataFrame["thickness"] = [thickness(depth,x,y) for (x,y) in zip(DataFrame["top"],DataFrame["bottom"])]
        q_increment = DataFrame["unit_weight"]*(DataFrame["bottom"]-DataFrame["top"])
        DataFrame["q"] = q_increment.cumsum() - q_increment/2
        DataFrame["self_weight"] = [self_weight(x,y) for (x,y) in zip(DataFrame["unit_weight"],DataFrame["thickness"])]
        DataFrame["alpha"] = [alpha(x,y) for (x,y) in zip(DataFrame["geology"],DataFrame["cu"])]
        DataFrame["qs_cal"] = [qs_cal(geology,spt,cu,alpha) for (geology,spt,cu,alpha) 
                       in zip(DataFrame["geology"],DataFrame["spt"],DataFrame["cu"],DataFrame["alpha"])]   
        DataFrame["rs_k"] = [rs_k(thickness,qs_cal) for (thickness,qs_cal) in zip(DataFrame["thickness"],DataFrame["qs_cal"])]
        DataFrame["qb_cal"] = [qb_cal(geology,spt,cu,q) for (geology,spt,cu,q) 
                     in zip(DataFrame["geology"],DataFrame["spt"],DataFrame["cu"],DataFrame["q"])]
        DataFrame["rb_k"] = [rb_k(depth,top,bottom,qb_cal) for (top,bottom,qb_cal) 
                     in zip(DataFrame["top"],DataFrame["bottom"],DataFrame["qb_cal"])]
    
        self_weight_t = DataFrame["self_weight"].sum()        
        rs_kt = DataFrame["rs_k"].sum()
        rb_kt = DataFrame["rb_k"].sum()
    
        if da1c1(rs_kt,rb_kt,self_weight_t) and da1c2(rs_kt,rb_kt,self_weight_t):
            #print(depth)
            return(depth)
            break
print(runtable())

18.6


In [90]:
print(DataFrame)

    top  bottom geology  spt  cu  unit_weight  thickness      q  self_weight  \
0   0.0     1.0    FILL    0   0           19        1.0    9.5        4.050   
1   1.0     3.0    FILL    0   0           19        2.0   38.0        8.100   
2   3.0    15.0       E    0  23           15       12.0  147.0       87.480   
3  15.0    17.8       E    0  27           15        2.8  258.0       20.412   
4  17.8    20.0       S   66   0           21        0.8  302.1        1.944   
5  20.0    23.0       S  100   0           21        0.0  356.7        0.000   
6  20.0    33.0       S  100   0           21        0.0  524.7        0.000   
7  33.0    36.0       S  100   0           21        0.0  692.7        0.000   
8  36.0    40.0       S  100   0           21        0.0  766.2        0.000   

   alpha  qs_cal     rs_k  qb_cal    rb_k  
0   0.00    0.00    0.000     0.0     0.0  
1   0.00    0.00    0.000     0.0     0.0  
2   1.00   23.00  736.000   354.0     0.0  
3   0.98   26.46  197.5

In [92]:
quantity = pd.read_csv("quantity.csv")
quantity_info = quantity[quantity["nospile"]==nospile]
df_out = pd.DataFrame(columns=['location','type','nos_pile1','length_pile1','nos_pile2','length_pile2','time','cost'])
new_dict = {}
new_dict['location'] = location
new_dict['type'] = piletype
new_dict['nos_pile1'] = int(quantity_info["nos1"])
gk = float(quantity_info["gk1"])
qk = float(quantity_info["gk1"])
new_dict['length_pile1']=runtable()
new_dict['nos_pile2'] = int(quantity_info["nos2"])
gk = float(quantity_info["gk2"])
qk = float(quantity_info["gk2"])
new_dict['length_pile2']=runtable()
total_length = new_dict['nos_pile1']*new_dict['length_pile1']+new_dict['nos_pile2']*new_dict['length_pile2']
total_volume = total_length * area
if piletype == "driven":
    new_dict['time'] = total_length // 0.6
    new_dict['cost'] = round(total_volume * 200000,2)
elif piletype == "bored":
    new_dict['time'] = total_length // 1.2
    new_dict['cost'] = round(total_volume * 300000,2)
df_out = df_out.append(new_dict,ignore_index = True)
print(df_out)

  location    type nos_pile1  length_pile1 nos_pile2  length_pile2   time  \
0        A  driven         2          20.1         2          20.1  134.0   

         cost  
0  13024800.0  
